# Import necessary libraries

In [1]:
import pandas as pd
import numpy as np

from bert_serving.client import BertClient
bc = BertClient()

import gensim.downloader as api
wv = api.load('word2vec-google-news-300')

C:\Users\Savannah\Anaconda3\lib\site-packages\smart_open\smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


# Load tag data

In [51]:
# Load tag data
transcriber_tags = pd.read_json('../data/project-5551-tags_2020-06-05.json')
expert_tags = pd.read_excel('../data/Unique_Zooniverse-tags (June2020).xlsx', converters={'name':str})

# Make sure they look correct
print("Transcriber Tags Preview:")
print("\tSize =", len(transcriber_tags))
display(transcriber_tags.head())

print("Expert Tags Preview:")
print("\tSize =", len(expert_tags))
display(expert_tags.head())

Transcriber Tags Preview:
	Size = 18719


,id,name,section,created_at,user_id,comment_id,taggable_id,taggable_type,project_id,user_login
0,808765,racerelations,project-5551,2018-05-08 20:30:33.679000+00:00,680853,1042125,15915024.0,Subject,5551,Timetrvlr
1,808766,segregation,project-5551,2018-05-08 20:30:33.683000+00:00,680853,1042125,15915024.0,Subject,5551,Timetrvlr
2,808822,job,project-5551,2018-05-08 21:37:05.433000+00:00,667271,1042241,15912210.0,Subject,5551,PepGiraffe
3,808823,governmentresponsibility,project-5551,2018-05-08 21:37:05.436000+00:00,667271,1042241,15912210.0,Subject,5551,PepGiraffe
4,808827,personalgrowth,project-5551,2018-05-08 21:43:37.837000+00:00,667271,1042252,15912677.0,Subject,5551,PepGiraffe


Expert Tags Preview:
	Size = 3963


,name
0,11airborn
1,15874180
2,15days
3,16005
4,17hours


In [52]:
print(transcriber_tags['name'].nunique())
print(len(transcriber_tags))
print(expert_tags['name'].nunique())
print(len(expert_tags))

3962
18719
3963
3963


# Determine best approximations for transcriber tags using expert tags

In [11]:
# NOTE: Consider adding spaces to multi-word tags

# Store results as a list of dictionaries
results = [None]*len(transcriber_tags)

# Track progress
count = 0

# Vectorize encoding function for better performance
def encode(tag):
    global count
    if count%100 == 0:
        print(count)
    count += 1
    return bc.encode([tag])[0]
v_encode = np.vectorize(encode)

# Get embeddings for tags
print("Check 1")
expert_embed = expert_tags['name'].apply(v_encode)
print("Check 2")
transcriber_embed = transcriber_tags['name'].apply(v_encode)
print("Check 3")

# List-comprehension alternative
#expert_embed = [bc.encode([i])[0] for i in expert_tags['name']]
#transcriber_embed = [bc.encode([i])[0] for i in transcriber_tags['name']]

Check 1
0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
Check 2
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
1820

In [37]:
# Compare each transcriber tag embedding to each expert tag embedding
count = 0

for i in range(len(transcriber_tags)):
    
    # Store best results
    result = {
        "transcriber_tag": transcriber_tags['name'].iloc[i],
        "best_expert_tag_w2v": None,
        "best_expert_tag_bert": None,
        "best_score_w2v": -1,
        "best_score_bert": -1
    }
    
    # Make comparisons
    for j in range(len(expert_tags)):
        #score_w2v = wv.similarity(transcriber_tags['name'].iloc[i], expert_tags['name'].iloc[j])
        score_bert = np.dot(transcriber_embed[i], expert_embed[j])/(np.linalg.norm(transcriber_embed[i])*np.linalg.norm(expert_embed[j]))
        
        # Update best approximations
        #if score_w2v > result.best_score_w2v:
        #    result.best_expert_tag_w2v = expert_tags['name'].iloc[j]
        #    result.best_score_w2v = score_w2v
        if score_bert > result['best_score_bert']:
            result['best_expert_tag_bert'] = expert_tags['name'].iloc[j]
            result['best_score_bert'] = score_bert
            
    # Add to results
    if count%100 == 0:
        print(count)
    count += 1
    results[i] = result

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
18

In [38]:
# Convert list of results to dataframe
df = pd.DataFrame(results)

# Write results to Excel

In [46]:
df = df.sort_values('best_score_bert')
# Sort results by cosine similarity first

# Cluster words that lack a close-enough approximation

In [47]:
df.head()

,transcriber_tag,best_expert_tag_w2v,best_expert_tag_bert,best_score_w2v,best_score_bert
10856,20739828,None,15874180,-1,0.927823
716,heknowsexactlywhathewantstodo,None,name:heknowsexactlywhathewantstodo,-1,0.963226
4634,soldierattitudes,None,soldierattitudes,-1,1.000000
13917,replacement,None,replacement,-1,1.000000
17458,newskills,None,newskills,-1,1.000000


In [ ]:
# Cluster transcriber words that don't have good expert matches
# kmeans, spectral